In [ ]:
import math, time, pprint
import geopandas as gpd
from shapely import geometry
import matplotlib.pyplot as plt
from scipy.spatial import (
    Voronoi,
    voronoi_plot_2d,
    Delaunay,
    delaunay_plot_2d,
    cKDTree
)
from sklearn.model_selection import KFold
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

import interpolators

plt.rcParams['figure.figsize'] = 15, 10

external_crs = "EPSG:4326"
internal_crs = "EPSG:3068"
berlin_districts = gpd.read_file("../shared/berlinDistricts.geojson")
measurements = gpd.read_file("meeting-test/data_2020-03-02T14-00-00.geojson")

berlin_districts = berlin_districts.to_crs(internal_crs)
measurements = measurements.to_crs(internal_crs)

x = np.array(measurements.geometry.x)
y = np.array(measurements.geometry.y)
values = np.array(measurements.value)
points = np.column_stack((x, y))

xmin, ymin, xmax, ymax = measurements.total_bounds
size = 500  # grid cell size in meters
xnew = np.linspace(xmin, xmax, int((xmax - xmin) / size))
ynew = np.linspace(ymin, ymax, int((ymax - ymin) / size))
zones = [0, 20, 40, 100, 200, 400]

In [ ]:
# Cross validation

def cross_validate(measurements_file):
    measurements = gpd.read_file(measurements_file)

    external_crs = "EPSG:4326"
    internal_crs = "EPSG:3068"

    measurements = measurements.to_crs(internal_crs)

    x = np.array(measurements.geometry.x)
    y = np.array(measurements.geometry.y)
    values = np.array(measurements.value)

    xmin, ymin, xmax, ymax = measurements.total_bounds
    size = 100  # grid cell size in meters
    xnew = np.linspace(xmin, xmax, int((xmax - xmin) / size))
    ynew = np.linspace(ymin, ymax, int((ymax - ymin) / size))

    interpolator_functions = {
        "nearest_neighbor": interpolators.nearest_neighbor,
        "discrete_natural_neighbor": interpolators.discrete_natural_neighbor,
        # "metpy_natural_neighbor": interpolators.metpy_natural_neighbor,
        # "scipy_natural_neighbor": interpolators.scipy_natural_neighbor,
        "idw": interpolators.inverse_distance_weighting,
        # "linear_barycentric": interpolators.linear_barycentric,
    }

    folds = 10
    seed = 987934
    kfold = KFold(folds, True, seed)
    avg_rmse_per_method = {}
    for method in interpolator_functions:
        interpolation_method = interpolator_functions[method] 
        sum_rmse = 0
        for train, test in kfold.split(values):
            train_points = np.column_stack((x[train], y[train]))
            train_values = values[train]

            interpolated_values = interpolation_method(x[test], y[test], train_points, train_values, grid=False)
            rmse = mean_squared_error(values[test], interpolated_values)
            sum_rmse +=rmse
        
        avg_rmse= sum_rmse/folds
        avg_rmse_per_method[method] = avg_rmse

    pprint.pprint(avg_rmse_per_method)


cross_validate("./validation2/data_2020-02-01T08-00-00.geojson")

In [ ]:
# Natural Neighbor method comparison

methods = {
    "discrete_natural_neighbor": interpolators.discrete_natural_neighbor,
    "metpy_natural_neighbor": interpolators.metpy_natural_neighbor,
    "scipy_natural_neighbor": interpolators.scipy_natural_neighbor
}

plt.rcParams["figure.figsize"] = 50, len(methods) * 30
plt.rcParams["font.size"] = 20
plt.rcParams["axes.titlesize"] = 50
plt.rcParams["axes.titlepad"] = 80

size = 500  # grid cell size in meters
xnew = np.linspace(xmin, xmax, int((xmax - xmin) / size))
ynew = np.linspace(ymin, ymax, int((ymax - ymin) / size))

fig, axes = plt.subplots(len(methods),1)

for i, ax in enumerate(axes):
    berlin_districts.boundary.plot(ax=ax, edgecolor="black")
    ax.set_title(f"Interpolation method: {list(methods.keys())[i]}")
    start = time.time()
    interpolated_values = list(methods.values())[i](xnew, ynew, points, values)
    end = time.time()
    plot = ax.contourf(xnew, ynew, interpolated_values, zones, cmap="winter_r")
    plot.cmap.set_under("w")
    plot.set_clim(zones[1])
    fig.colorbar(plot, ax=ax)
    print(f"Interpolation method: {list(methods.keys())[i]} -> {end-start}")

fig.savefig("natural-neighbor-comparison.png")

In [ ]:
# Grid comparison

interpolation_method = interpolators.discrete_natural_neighbor

# cell_sizes = [5000, 1000, 500, 300, 100, 50, 10]
cell_sizes = [5000, 1000, 500, 300, 100]

plt.rcParams["figure.figsize"] = 50, len(cell_sizes) * 30
plt.rcParams["font.size"] = 20
plt.rcParams["axes.titlesize"] = 50
plt.rcParams["axes.titlepad"] = 80

fig, axes = plt.subplots(len(cell_sizes),1)

for i, ax in enumerate(axes):
    berlin_districts.boundary.plot(ax=ax, edgecolor="black")
    size = cell_sizes[i]  # grid cell size in meters
    xnew = np.linspace(xmin, xmax, int((xmax - xmin) / size))
    ynew = np.linspace(ymin, ymax, int((ymax - ymin) / size))
    ax.set_title(f"Grid cell size = {size}m")
    start = time.time()
    interpolated_values = interpolation_method(xnew, ynew, points, values)
    end = time.time()
    plot = ax.contourf(xnew, ynew, interpolated_values, zones, cmap="winter_r")
    plot.cmap.set_under("w")
    plot.set_clim(zones[1])
    fig.colorbar(plot, ax=ax)
    print(f"Grid cell size = {size}m -> {end-start}")

fig.savefig(f"grid-comparison-{interpolation_method.__name__}.png")